<a href="https://colab.research.google.com/github/vifirsanova/compling/blob/main/projects/llm_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Псевдокод для проектов с LLM

1. инициализация клиента API

In [ ]:
from huggingface_hub import InferenceClient

In [ ]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"

In [ ]:
client = InferenceClient(model_name, token='ваш токен здесь')

2. создание функции для запросов к модели

In [ ]:
def llm_inference(user_sample):
  output = client.chat.completions.create(
          messages=[
              {"role": "system", "content": "you are natural language processing guide\n"
                                            "explain the provided topic\n"
              },
              {"role": "user",
              "content": f"explain the basics of {user_sample}"},
          ],
          stream=False,
          max_tokens=128,
          temperature=0.7,
          top_p=0.1
          )
  return output.choices[0].get('message')['content']

Примеры функций:

- функция с промптом для генерации случайных текстов, которые нужно перевести
- функция принимает на вход задачу на перевод текста и перевод --> используем промпт для оценки качества перевода
- функция принимает на вход описание темы для генерации задач на грамматику со множественным выбором

3. создание функции с возможностью парсинга словаря (json-объекта)

In [ ]:
# формат ответа
response_format = {
    "type": "json",
    "value": {
        "properties": {
            "concept_name": {"type": "string"},
            "concept_description": {"type": "string"},
        },
        "required": ["concept_name", "concept_description",]
    },
}

# ответ
response = client.chat_completion(
    messages=
              {"role": "system", "content": "you are natural language processing guide\n"
                                            "explain the provided topic\n"
              },
              {"role": "user",
              "content": f"explain the basics of {user_sample}"},
          ],
    response_format=response_format,
    max_tokens=500,)
response.choices[0].message.content

После генерации json ответ нужно спарсить, превратить его в настоящий, рабочий словарь

In [ ]:
import ast

result = ast.literal_eval(response.choices[0].message.content)

В этом примере мы добавили `response_format`, чтобы задать ограничение на генерацию словаря (объекта json)

`response_format` можно адаптировать, чтобы генерировать условия задач и их решения (варианты ответов)

4. разработка функций для проверки ответов пользователя

- здесь вы можете использовать средства нативного Python (например, условия и циклы), чтобы проверить, совпал ли ответ пользователя с правильным ответом из объекта json